# IBCS Dashboard Compliance – Model Export

- Export trained **ConvNeXt**, **MobileNet**, and **SVM** models into deployable formats.
- Implement and test image/PDF preprocessing (resize, normalize, etc.).
- Run merged models and return a JSON-like output (`label + confidence`) that can later be used by the backend.

#### Conclusion
I tested megred models with Not Compliant dashboard picture, my expected predict result: "Not Compliant"

ConvNeXt: [0.46 Compliant, 0.54 Not Compliant] -> Predicts Not Compliant

MobileNet: [0.78 Compliant, 0.22 Not Compliant] -> Predicts Compliant

SVM: [0.70 Compliant, 0.30 Not Compliant] -> Predicts Compliant

Ensemble: [0.61 Compliant, 0.39 Not Compliant] -> Predicts Compliant

Based on current results, the ensemble predicts "Compliant" because MobileNet and SVM both strongly favor this class. To get "Not Compliant" as the final result, should:

- Find suitable ensemble weights to give more importance to ConvNeXt

- Use majority voting (though this would still give "Compliant" with current predictions)

My current weights (0.5, 0.4, 0.1) give too much influence to MobileNet and SVM, which are both predicting "Compliant" with high confidence, overwhelming ConvNeXt's "Not Compliant" prediction.
Until when I changed weights (0.9, 0.05, 0.05), the result changed to "Not Compliant" prediction. 




In [4]:
# %pip install pdf2image

## 1. Setup & Imports

In [21]:
# Core libraries
import os
import json

import torch
from torchvision import models, transforms
import torch.nn as nn
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.svm import SVC
import numpy as np
from PIL import Image
import torchvision.transforms as T


import numpy as np
from PIL import Image
from pdf2image import convert_from_path

import joblib
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


## 2. Load Trained Models (for Export)

In this section, loading already trained models (ConvNeXt, MobileNet, SVM) with the correct architectures and weights.


In [15]:

device = "cuda" if torch.cuda.is_available() else "cpu"

CONVNEXT_CKPT_PATH = "Checkpoints/convnext.pt"
convnext = models.convnext_tiny(weights=None) 
if os.path.exists(CONVNEXT_CKPT_PATH):
    print("Found checkpoint at:", CONVNEXT_CKPT_PATH)
    state = torch.load(CONVNEXT_CKPT_PATH, map_location="cpu")
    convnext.load_state_dict(state) 
    print("Pretrained weights loaded")

    # Because the imported path has 1000 output classes head
    # Now i need 2-class model so replace classifier with a 2-class head
    num_classes = 2
    in_features = convnext.classifier[2].in_features
    convnext.classifier[2] = nn.Linear(in_features, num_classes)

    convnext = convnext.to(device).eval()
    print("ConvNeXt now has 2 output classes")
else:
    print("ConvNeXt checkpoint not found.")

Found checkpoint at: Checkpoints/convnext.pt
Pretrained weights loaded
ConvNeXt now has 2 output classes


In [25]:

# Feature extractor (for SVM)
convnext_feat = models.convnext_tiny(weights="IMAGENET1K_V1")
convnext_feat.classifier[2] = nn.Identity()  # so output is the 768-d embedding
convnext_feat = convnext_feat.to(device).eval()

# Classifier model (2 classes, fine-tuned)
device = "cuda" if torch.cuda.is_available() else "cpu"

CONVNEXT_CKPT_PATH = "Checkpoints/convnext.pt"
convnext_cls = models.convnext_tiny(weights=None) 
if os.path.exists(CONVNEXT_CKPT_PATH):
    print("Found checkpoint at:", CONVNEXT_CKPT_PATH)
    state = torch.load(CONVNEXT_CKPT_PATH, map_location="cpu")
    convnext_cls.load_state_dict(state) 
    print("Pretrained weights loaded")

    # Because the imported path has 1000 output classes head
    # Now i need 2-class model so replace classifier with a 2-class head
    num_classes = 2
    in_features = convnext_cls.classifier[2].in_features
    convnext_cls.classifier[2] = nn.Linear(in_features, num_classes)

    convnext_cls = convnext_cls.to(device).eval()
    print("ConvNeXt now has 2 output classes")
else:
    print("ConvNeXt checkpoint not found.")

Found checkpoint at: Checkpoints/convnext.pt
Pretrained weights loaded
ConvNeXt now has 2 output classes


In [41]:
MOBILENET_KERAS    = "Checkpoints/mobilenet.keras"
SVM_CONVNEXT = "Checkpoints/svm_convnext.pkl"
SVM_SCALER = "Checkpoints/svm_scaler.pkl"
# SVM_PATH = "Checkpoints/svm.pkl"

mobilenet_keras = load_model(MOBILENET_KERAS)
mobilenet_keras.summary()

svm_scaler = joblib.load(SVM_SCALER)
svm_convnext = joblib.load(SVM_CONVNEXT)
print("SVM loaded:", svm_scaler)
print("SVM ConvNeXt:", svm_convnext)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,261,829 (8.63 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2,564 (10.02 KB)

SVM loaded: StandardScaler()
SVM ConvNeXt: SVC(probability=True, random_state=42)


## 3. Export Models to Deployable Formats

Here we export:

- ConvNeXt → PyTorch**TorchScript** (`convnext_ts.pt`)
- MobileNet → **Keras** model (`mobilenet_ts.keras`)
- SVM → **joblib** (`svm.pkl`)

These files can be used for backend

In [27]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

pt_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

def load_and_resize(path, size=224):
    img = Image.open(path).convert("RGB")
    img = img.resize((size, size))
    return img

def preprocess_for_convnext(img):
    "PyTorch tensor (1,3,224,224)"
    tensor = pt_transform(img)                 # (3,224,224)
    return tensor.unsqueeze(0)                 # (1,3,224,224)

def preprocess_for_mobilenet(img):
    "Keras array (1,224,224,3), MobileNetV2 style"
    arr = np.array(img).astype("float32")
    arr = arr / 255.0                         # [0,1]
    return np.expand_dims(arr, axis=0)        # (1,224,224,3)
    

After setting up fuctions for ConvNeXt and MobileNet, define features for SVM traning. First I need to use ConvNeXt as a feature extractor

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
def extract_features_convnext(image_input):
    if isinstance(image_input, str):
        img = Image.open(image_input).convert("RGB")
    elif isinstance(image_input, np.ndarray):
        if image_input.ndim == 2:  # grayscale (H, W)
            # convert to 3-channel by duplicating
            image_input = np.stack([image_input] * 3, axis=-1)
        elif image_input.ndim == 3 and image_input.shape[2] == 1:  
            image_input = np.repeat(image_input, 3, axis=2)
        
        img = Image.fromarray(image_input.astype(np.uint8))
    
    x = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = convnext_feat(x)
    return features.cpu().numpy().flatten()



In [57]:
label = {0: "Compliant", 1: "Not Compliant"}

def predict_all_models(image_path):
    img = load_and_resize(image_path, size=224)
    # ConvNeXt
    x_pt = preprocess_for_convnext(img).to(device)
    with torch.no_grad():
        logits = convnext_cls(x_pt) # the model outputs raw scores (called logits), shape (1, 2) since you have 2 classes
        probs_pt = torch.softmax(logits, dim=1)[0].cpu().numpy() # eg the softmax return: [0.2, 0.8] → means 20% Compliant, 80% Not Compliant.

    # MobileNet
    x_keras = preprocess_for_mobilenet(img)
    prob_mbn_1 = float(mobilenet_keras.predict(x_keras, verbose=0)[0][0]) #eg prob_mbn_1 return 22% -> 22% Not Compliant
    probs_mbn = np.array([1 - prob_mbn_1, prob_mbn_1])

    # SVM
    feat = extract_features_convnext(image_path) 
    feat_2d = feat.reshape(1, -1)    
    feat_scaled = svm_scaler.transform(feat_2d) 
    svm_probs = svm_convnext.predict_proba(feat_scaled)[0] 


    # Ensemble (calculate sum of 3 models)
    w_pt, w_mbn, w_svm = 0.5, 0.4, 0.1
    ensemble_probs = w_pt*probs_pt + w_mbn*probs_mbn + w_svm*svm_probs
    ensemble_probs = ensemble_probs / ensemble_probs.sum()

    pred_idx = int(np.argmax(ensemble_probs)) 
    pred_label = label[pred_idx]
    confidence = float(ensemble_probs[pred_idx])

    # ConvNeXt + MobileNet
    w_pt, w_mbn= 0.8, 0.2
    mixed_probs = w_pt*probs_pt + w_mbn*probs_mbn
    mixed_probs = mixed_probs / mixed_probs.sum()

    pred_idx = int(np.argmax(mixed_probs)) 
    pred_label = label[pred_idx]
    confidence = float(mixed_probs[pred_idx])

    return {
        "label": pred_label,
        "confidence": confidence,
        "probs": {
            "ConvNeXt": probs_pt.tolist(),
            "MobileNet": probs_mbn.tolist(),
            "SVM": svm_probs.tolist(),
            "Ensemble": ensemble_probs.tolist(),
            "Mixed (ConvNeXt + MobileNet)": mixed_probs.tolist()
        }
    }

result = predict_all_models("example.png")
print(json.dumps(result, indent=2))



{
  "label": "Compliant",
  "confidence": 0.5280108210085532,
  "probs": {
    "ConvNeXt": [
      0.4646602272987366,
      0.5353397727012634
    ],
    "MobileNet": [
      0.7814132273197174,
      0.2185867726802826
    ],
    "SVM": [
      0.6993136590104901,
      0.30068634098950975
    ],
    "Ensemble": [
      0.6148267704783043,
      0.38517322952169575
    ],
    "Mixed (ConvNeXt + MobileNet)": [
      0.5280108210085532,
      0.47198917899144677
    ]
  }
}
